In [1]:
import pandas as pd
import glob
import numpy as np
from datetime import datetime
import datetime as dt
import csv

now = datetime.now()
format = "%m%d%Y"
time1 = now.strftime(format)

print("Import complete")

Import complete


In [2]:
def Grouper(df,column,criteria):
    column = column
    grouped = df.groupby(df[column])
    g = grouped.get_group(criteria)
    return g

def InferName(folder,doc):
    files = [file.split("\\")[-1].split(".")[0] for file in glob.glob(folder)]
    char = ["_","-","/"]
    splitter = "None"
    for c in char:
        countl = [f for f in files if c in f]
        if len(countl) == len(files):
            splitter = c
    if splitter == "None":
        return doc, splitter
    else:
        ind = "None"
        flist = [f.split(splitter) for f in files]
        for i in range(0,len(flist[0])):
            ilist = list(set([f[i] for f in flist]))
            if len(ilist) == len(flist):
                ind = i
        if ind == "None":
            return doc
        else:
            dname = doc.split("\\")[-1].split(".")[0].split(splitter)[ind]
            return dname
        
print("Modules Loaded")

Modules Loaded


In [3]:
def SplitSource(file):
    with open(file) as f:
        reader = csv.reader(f)
        S = [row for row in reader]
    user = []
    master = []
    for line in S:
        user.append(line)
        if '</user>' in line[0]:
            master.append(user)
            user = []
    return master

def GetDomain(file):
    domain = file.split("\\")[-1].split("_")[0]
    return domain

def ChooseGroups(users):
    groups = [entry for entry in users if 'user group="" admin="false" dn=":GROUP:' in entry[0][0]]
    fullperms = []
    for g in groups:
        groupname = g[0][0].split(':')[-1].replace('">','')
        g.pop()
        perms = []
        for line in g[2:]:
            try:
                perm = line[0].split('name=')[-1].split('<')[0].split(">")
                name = perm[0].split('"')[1]
                level = perm[-1]
                perms.append([name,level])
            except:
                print("Error with:",line)
        fullperms.append([groupname,perms])
    return fullperms
        
def CreateMasterList(users):
    fullperms = ChooseGroups(users)
    groups = [g[0] for g in fullperms]
    data = [g[1] for g in fullperms]
    unames = []
    for d in data:
        for item in d:
            if item[0] not in unames:
                unames.append(item[0])
    masterlist = []
    for n in unames:
        pg = [n]
        for d in data:
            names = [item[0] for item in d]
            if n not in names:
                pg.append('0 - None')
            else:
                for line in d:
                    if line[0] == n:
                        if line[1] == '0':
                            pg.append("0 - None")
                        elif line[1] == '1':
                            pg.append("1 - View")
                        elif line[1] == '2':
                            pg.append("2 - View/Edit")
                        elif line[1] == '3':
                            pg.append("3 - View/Edit/Add/Delete")
                        else:
                            pg.append(line[1])
        masterlist.append(pg)
    return masterlist, groups

def GetPermsMatrix(file):
    domain = GetDomain(file)
    users = SplitSource(file)
    ML, groups = CreateMasterList(users)
    columns = ["Product","Category","Permission"] + groups
    master = []
    for line in ML:
        names = line.pop(0)
        PCP = names.split("/")[1:]
        newline = PCP+line
        master.append(newline)
    hrdf = pd.DataFrame(master,columns=columns)
    return hrdf, domain

print("Module Loaded")

Module Loaded


In [4]:
with pd.ExcelWriter('Reports/1.HumanReadablePermissions.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            hrdf, domain = GetPermsMatrix(file)
            hrdf.to_excel(writer, sheet_name=domain,index=False)
            
print("Permissions Loaded to File")

Permissions Loaded to File


In [50]:
def GetCompareMatrix(file):
    comparedf, comparedomain = GetPermsMatrix(file)
    ind = ["Product","Category","Permission"]
    groups = [col for col in masterdf.columns if col not in ind]
    m = []
    for col in groups:
        try:
            newcol = str(col+" - Update")
            comparedf[newcol] = comparedf[col]
            comparedfa = comparedf[ind + [newcol]]
            masterdfa = masterdf[ind + [col]]
            matrix = masterdfa.merge(comparedfa, on=ind)
            matrix['Update'] = np.where(matrix[col] != matrix[newcol],0,1)
            matrix = Grouper(matrix,'Update',0)
            matrix = matrix[ind + [col]]
            m.append(matrix)
        except:
            pass

    mast = masterdf[ind]
    x = 1
    while x < len(m):
        c = m[x]
        c.reset_index()
        mast = mast.merge(c,how='outer')
        x += 1
    permcols = [col for col in mast.columns if col not in ind]
    mast = mast.dropna(how='all',subset=permcols)
    return mast

print("Module Loaded")

Module Loaded


In [51]:
md = 'peapp1'

for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1] and GetDomain(file) == md:
            masterdf, domain = GetPermsMatrix(file)
with pd.ExcelWriter('Reports/2.ComparePermissionsMatrix.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            comparedf, comparedomain = GetPermsMatrix(file)
            mast = GetCompareMatrix(file)
            mast.to_excel(writer, sheet_name=comparedomain,index=False)

print("Permissions Loaded to File")

Permissions Loaded to File


In [52]:
def GetUsers(file):
    domain = GetDomain(file)
    groups = SplitSource(file)
    users = [entry for entry in groups if 'user group="|USR|' in entry[0][0]]
    master = []
    for entry in users:
        u = ['User']
        name = "None"
        for line in entry:
            if 'Full Name' in line[0]:
                name = line[0].split(">")[1].split("<")[0]
        u.append(name)
        roles = []
        header = entry[0]
        if len(header) == 1:
            al = header[0].split(" ")
        else:
            roles.append(header[0].split('"')[-1])
            x = 3
            while x <= len(header):
                roles.append(header[x-2])
                x += 1
            al = header[-1].split(" ")
        for item in al:
            if 'dn=' in item:
                un = item.split('"')[1]
                u.append(un)
        if 'admin="true"' in al:
            u.append('Yes')
        else:
            u.append("No")
        for r in al:
            if '|USR|' in r:
                rs = [item for item in r.split('"') if '|USR|' in item]
                for item in rs:
                    roles.append(item)
        u.append(roles)
        master.append(u)
        
    consoles = [entry for entry in groups if 'user group="|CON|' in entry[0][0]]
    for entry in consoles:
        u = ['Console']
        name = "None"
        for line in entry:
            if 'Description' in line[0]:
                name = line[0].split(">")[1].split("<")[0]
        u.append(name)
        roles = []
        header = entry[0]
        if len(header) == 1:
            al = header[0].split(" ")
        else:
            roles.append(header[0].split('"')[-1])
            x = 3
            while x <= len(header):
                roles.append(header[x-2])
                x += 1
            al = header[-1].split(" ")
        for item in al:
            if 'dn=' in item:
                un = item.split(':')[-1].split('"')[0]
                u.append(un)
        if 'admin="true"' in al:
            u.append('Yes')
        else:
            u.append("No")
        for r in al:
            if '|CON|' in r:
                rs = [item for item in r.split('"') if '|CON|' in item]
                for item in rs:
                    roles.append(item)
        u.append(roles)
        master.append(u)
    df = pd.DataFrame(master,columns=['Type','Assigned To','Name','Admin?','Groups'])
                
                    
    return df, domain

def ConsoleUserRole(file):
    df, domain = GetUsers(file)
    user = Grouper(df,'Type','User')
    console = Grouper(df,'Type','Console')
    console = console[console['Assigned To'] != '']
    cur = console.merge(user,on=['Assigned To'],how='inner')
    return cur, domain
        
print("Module Loaded")

Module Loaded


In [53]:
with pd.ExcelWriter('Reports/3.UsersAndConsoles.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            df, domain = GetUsers(file)
            df.to_excel(writer, sheet_name=domain,index=False)

with pd.ExcelWriter('Reports/4.UsersAndConsoles - All Domains.xlsx') as writer:
    masterlist = []
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            df, domain = GetUsers(file)
            df['Domain'] = domain
            masterlist.append(df)
    df = pd.concat(masterlist)
    data = df.values.tolist()
    N = list(set([item[2] for item in data]))
    ml = []
    for n in N:
        alist = list(set([line[1] for line in data if line[2] == n]))
        tlist = list(set([line[0] for line in data if line[2] == n]))
        dlist = list(set([line[5] for line in data if line[2] == n]))
        glist = []
        for line in data:
            if line[2] == n:
                for item in line[4]:
                    if item not in glist:
                        glist.append(item)
                
        ml.append([n,alist,tlist,dlist,glist])
    df = pd.DataFrame(ml,columns=['Name','Assigned To','Type','Domains','Groups'])
    df.to_excel(writer, sheet_name="All",index=False)
            
print("Permissions Loaded to File")

Permissions Loaded to File


In [54]:
masterlist = []
for file in glob.glob('uds/all_UDS/*'):
    if "uds.xml" in file.split("\\")[-1]:
        df, domain = GetUsers(file)
        df['Domain'] = domain
        masterlist.append(df)
df = pd.concat(masterlist)
print(df.shape)

(12915, 6)


In [55]:
#print(df.columns)
#data = df.values.tolist()
#print(len(data))
#N = list(set([item[2] for item in data]))
#l = []
#for n in N:
    #alist = list(set([line[1] for line in data if line[2] == n]))
    #tlist = list(set([line[0] for line in data if line[2] == n]))
    #dlist = list(set([line[5] for line in data if line[2] == n]))
    #ml.append([n,alist,tlist,dlist])
#df = pd.DataFrame(master,columns=['Name','Assigned To','Type','Domains'])


In [56]:
with pd.ExcelWriter('Reports/5.ConsoleUserRole.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            cur, domain = ConsoleUserRole(file)
            cur.to_excel(writer, sheet_name=domain,index=False)
            
print("Permissions Loaded to File")

Permissions Loaded to File


In [57]:
def GetRoleDomainMatrix(file):
    domain = GetDomain(file)
    users = SplitSource(file)
    groups = [entry for entry in users if 'user group="" admin="false" dn=":GROUP:' in entry[0][0]]
    gnames = [g[0][0].split(':')[-1].replace('">','') for g in groups]
    return gnames, domain


allgroups = []
with pd.ExcelWriter('Reports/6.RoleDomainMatrix.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            groups, domain = GetRoleDomainMatrix(file)
            for g in groups:
                if g not in allgroups:
                    allgroups.append(g)
    allgroups = sorted(allgroups)
    df = pd.DataFrame(allgroups,columns=['Group'])
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            glist = []
            groups, domain = GetRoleDomainMatrix(file)
            for g in allgroups:
                if g in groups:
                    glist.append("Yes")
                else:
                    glist.append("No")
            df[domain] = glist
                
    df.to_excel(writer, sheet_name='RoleDomainMatrix',index=False)
            
print("Permissions Loaded to File")

Permissions Loaded to File


## User Domain Matrix

In [58]:
def GetUserDomainMatrix(file):
    df, domain = GetUsers(file)
    user = Grouper(df,'Type','User')
    return user, domain


allusers = []
with pd.ExcelWriter('Reports/7.UserDomainMatrix.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Assigned To'] +" - " + user['Name']
            ulist = user['UName'].tolist()
            for u in ulist:
                if u not in allusers:
                    allusers.append(u)
    allusers = sorted(allusers)
    df = pd.DataFrame(allusers,columns=['User'])
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            ulist = []
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Assigned To'] +" - " + user['Name']
            for u in allusers:
                groups = user.loc[user['UName'] == u, 'Groups'].tolist()
                if groups != []:
                    ulist.append(groups)
                else:
                    ulist.append("")
                    

            df[domain] = ulist
                
    df.to_excel(writer, sheet_name="UserDomainMatrix",index=False)
    
    allusers = []
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Name'] +" - " + user['Assigned To']
            ulist = user['UName'].tolist()
            for u in ulist:
                if u not in allusers:
                    allusers.append(u)
    allusers = sorted(allusers)
    df = pd.DataFrame(allusers,columns=['User'])
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            ulist = []
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Name'] +" - " + user['Assigned To']
            for u in allusers:
                groups = user.loc[user['UName'] == u, 'Groups'].tolist()
                if groups != []:
                    ulist.append(groups)
                else:
                    ulist.append("")
                    

            df[domain] = ulist
                
    df.to_excel(writer, sheet_name="UserDomainMatrix - Inverse",index=False)
            
print("Permissions Loaded to File")

Permissions Loaded to File


## Console Domain Matrix

In [59]:
def GetUserDomainMatrix(file):
    df, domain = GetUsers(file)
    user = Grouper(df,'Type','Console')
    return user, domain


allusers = []
with pd.ExcelWriter('Reports/8.ConsoleDomainMatrix.xlsx') as writer:
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Assigned To'] +" - " + user['Name']
            ulist = user['UName'].tolist()
            for u in ulist:
                if u not in allusers:
                    allusers.append(u)
    allusers = sorted(allusers)
    df = pd.DataFrame(allusers,columns=['Console'])
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            ulist = []
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Assigned To'] +" - " + user['Name']
            for u in allusers:
                groups = user.loc[user['UName'] == u, 'Groups'].tolist()
                if groups != []:
                    ulist.append(groups)
                else:
                    ulist.append("")
                    

            df[domain] = ulist
                
    df.to_excel(writer, sheet_name="ConsoleDomainMatrix",index=False)
    
    allusers = []
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Name'] +" - " + user['Assigned To']
            ulist = user['UName'].tolist()
            for u in ulist:
                if u not in allusers:
                    allusers.append(u)
    allusers = sorted(allusers)
    df = pd.DataFrame(allusers,columns=['Console'])
    for file in glob.glob('uds/all_UDS/*'):
        if "uds.xml" in file.split("\\")[-1]:
            ulist = []
            user, domain = GetUserDomainMatrix(file)
            user['UName'] = user['Name'] +" - " + user['Assigned To']
            for u in allusers:
                groups = user.loc[user['UName'] == u, 'Groups'].tolist()
                if groups != []:
                    ulist.append(groups)
                else:
                    ulist.append("")
                    

            df[domain] = ulist
                
    df.to_excel(writer, sheet_name="ConsoleDomainMatrix - Inverse",index=False)
            
print("Permissions Loaded to File")

Permissions Loaded to File


In [60]:
ag = [g for g in allgroups if 'CON' not in g]
with pd.ExcelWriter('Reports/9.UserDomainMatrix - by Role.xlsx') as writer:
    for gr in ag:
        g = gr.split("|")[-1].split("_")[0]
        allusers = []
        for file in glob.glob('uds/all_UDS/*'):
            if "uds.xml" in file.split("\\")[-1]:
                userraw, domain = GetUserDomainMatrix(file)
                user = userraw[userraw['Groups'].astype(str).str.contains(g)==True]
                user['UName'] = user['Assigned To'] +" - " + user['Name']
                ulist = user['UName'].tolist()
                for u in ulist:
                    if u not in allusers:
                        allusers.append(u)
        allusers = sorted(allusers)
        df = pd.DataFrame(allusers,columns=['User'])
        for file in glob.glob('uds/all_UDS/*'):
            if "uds.xml" in file.split("\\")[-1]:
                ulist = []
                userraw, domain = GetUserDomainMatrix(file)
                user = userraw[userraw['Groups'].astype(str).str.contains(g)]
                user['UName'] = user['Assigned To'] +" - " + user['Name']
                for u in allusers:
                    groups = user.loc[user['UName'] == u, 'Groups'].tolist()
                    if groups != []:
                        ulist.append(groups)
                    else:
                        ulist.append("")
                    

                df[domain] = ulist
                
        try:
            df.to_excel(writer, sheet_name=gr,index=False)
        except:
            pass
            
print("Permissions Loaded to File")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Permissions Loaded to File


In [61]:
def GetCIPAccess(file):
    with open(file) as f:
        reader = csv.reader(f)
        S = [row for row in reader]
        R = list(set([item[0] for item in S]))
        access = list(set([item[2] for item in S]))
        mlist = []
        for r in R:
            nlist = list(set([item[1] for item in S if item[0] == r]))
            alist = list(set([item[2] for item in S if item[0] == r]))
            mlist.append([r,nlist,alist])
    return mlist, access
        
directory = 'PAM\*'
Target = max([InferName(directory,f) for f in glob.glob(directory) if 'AllUser' in f])
for file in glob.glob(directory):
    domain = InferName(directory, file)
    if domain == Target:
        CIPA, access = GetCIPAccess(file)

Domain_Dict = {
    'TMS':['prod','ldan1','pdac1','bdac2','edan1','edan2','tdan2','wdan1','ldan2','bdac1',
           'pdac2','tdan1','wdan2','bapp1','bapp2','bchron1','bchron2','biccp1','biccp2','binter1','binter2',
           'brchron1','brchron2','papp1','papp2','pchron1','pchron2','piccp1','piccp2','pinter1','pinter2',
           'prchron1','prchron2'],
    'QAS':['pqdac2','eqasdan1','tqasdan1','wqasdan2','bqdac2','pqdac1', 'wqasdan1','bqdac1','tqasdan2','lqasdan1',
           'eqasdan2','lqasdan2','bqapp1','bqapp2','bqchron1','bqchron2','bqiccp1','bqiccp2','bqinter1','bqinter2',
           'bqrchron1','bqrchron2','pqapp1','pqapp2','pqchron1','pqchron2','pqiccp1','pqiccp2','pqinter1','pqinter2',
           'pqrchron1','pqrchron2',],
    'ENG':['peapp1'],
    'OTS':['pots2','pots1'],
    'PDS':['epdan2','wpdan1', 'lpdan2','lpdan1','wpdan2','epdan1','tpdan2','tpdan1','ppds1','bpds1'],
    'REMOTE':['tstudy1','wstudy1','pqstudy1','pstudy1','bstudy1','estudy1','bstudy2','pstudy2','bqstudy1','tqasstudy1',
              'lstudy1', 'bqstudy2','pqstudy2']
}

server_dict = {}
directory = 'uds/all_UDS/*'
for file in glob.glob('uds/all_UDS/*'):
    server = InferName(directory,file)
    domain = "None"
    for key, value in Domain_Dict.items():
        if server in value:
            domain = key
    server_dict.update({server:domain})

OSI_U = []
for file in glob.glob('uds/all_UDS/*'):
    df, server = GetUsers(file)
    df['Server'] = server
    df['Domain'] = server_dict.get(server)
    OSI = df.values.tolist()
    OSI_Users = [line for line in OSI if line[0]=='User']
    for line in OSI_Users:
        OSI_U.append(line)

file = 'Analysis/ReferenceData/PAM Access Match.csv'
with open(file) as f:
    reader = csv.reader(f)
    Match = [row for row in reader]
    RACF = [m[1] for m in Match]
    UN = [m[0] for m in Match]

print("Users that Need to be Added to the Match File")
for line in OSI_U:
    r = [m[1] for m in Match if m[0]==line[2]]
    if len(r) == 0:
        print(line)
print("Validation Complete")

print("Users that have multiple entries in the Match File")
for line in OSI_U:
    r = [m[1] for m in Match if m[0]==line[2]]
    if len(r) > 1:
        print(line)
print("Validation Complete")

######

NFED = []
IU = []
IUNC = []
CIPR = [item[0] for item in CIPA]
for line in OSI_U:
    r = [m[1] for m in Match if m[0]==line[2]]
    if r == []:
        continue
    try:
        line.append(r[0])
    except:
        print(line)
    if r[0] == 'Not found in employee directory':
        NFED.append(line)
    elif r[0] not in CIPR:
        IUNC.append(line)
    else:
        IU.append(line)
        

print("Identified Users:",len(IU))
print("Identified Users, Not CIP:",len(IUNC))
print("Not Found in Employee Directory:",len(NFED))

#####

file = 'Analysis\Approved_Access_03.2021\Approved_Access_Consolidated_Feedback.xlsx'
AL_RbR = []
ALDF = pd.read_excel(file, sheet_name='Feedback',na_values = '')
ALDF['domain'] = 'TMS'
AL = ALDF.values.tolist()



A = list(set([line[4] for line in AL]))

for RACF in A:
    newroles = [line[5] for line in AL if line[4]==RACF]
    newline = [RACF,newroles]
    AL_RbR.append(newline)

for line in IU:
    CA = [c[2] for c in CIPA if c[0]==line[-1]]
    line.append(CA)
    AL = [r[1] for r in AL_RbR if r[0]==line[-2]]
    line.append(AL)



print("Module Loaded")

Users that Need to be Added to the Match File
['User', 'Simeon Milton', 'smilton', 'No', ['|USR|DOC_Operator'], 'pdac2', 'TMS']
['User', 'Zachary Leonard', 'zleonard', 'No', ['|USR|DOC_Operator'], 'pdac2', 'TMS']
['User', 'Jonathon Szabo', 'jszabo', 'No', ['|USR|TGM_Operator'], 'pdac2', 'TMS']
['User', 'Simeon Milton', 'smilton', 'No', ['|USR|DOC_Operator'], 'prod', 'TMS']
['User', 'Zachary Leonard', 'zleonard', 'No', ['|USR|DOC_Operator'], 'prod', 'TMS']
['User', 'Jonathon Szabo', 'jszabo', 'No', ['|USR|TGM_Operator'], 'prod', 'TMS']
Validation Complete
Users that have multiple entries in the Match File
Validation Complete
Identified Users: 4552
Identified Users, Not CIP: 177
Not Found in Employee Directory: 415
Module Loaded


In [62]:
TMS_Access = ['Field (ODV)','Database (SCADA)','Integration','TGM Supervisor','State Estimator(TNA)','Planning','DOC Dispatcher'
              'DOC Supervisor','Communications (RTU)','Support','TGM Dispatcher','TGO Supervisor']
siemenssubs = {
        "|USR|Database_(SCADA)": "Database (SCADA)",
        "|USR|Integration": "Integration",
        "|USR|State_Estimator(TNA)": "State Estimator(TNA)",
        "|USR|DOC_Operator": "DOC Dispatcher",
        "|USR|Communications_(RTU)": "Communications (RTU)",
        "|USR|TGM_Operator": "TGM Dispatcher",
        "N/A": "Planning",
        "N/A": "DOC Supervisor",
        "N/A": "Support",
        "N/A": "TGO Supervisor",
        "N/A": "Field (ODV)"
}

['Type','Name','UserName','Admin?','OSIAccess','server','domain','RACF','CIPAccess','ApprovedRole']
IUV = []
for u in IU:
    for r in u[4]:
        siemensrole = siemenssubs.get(r,r)
        newline = [item for item in u[0:4]] + [r] + [item for item in u[5:8]]
        if siemensrole in u[8][0]:
            newline.append(siemensrole)
            newline.append("Success")
        else:
            newline.append("Access Not Found")
            newline.append("Failure")
        try:
            if r in u[9][0]:
                newline.append(r)
                newline.append("Success")
            else:
                newline.append("Access Not Found")
                newline.append("Failure")
        except:
            newline.append("Not Originally Approved for Access")
            newline.append("Failure")
        if newline[9] == "Success" and newline[11] == "Success":
            newline.append("None")
        elif newline[11] == "Success" and newline[9] != "Success":
            newline.append("True-up PAM")
        else:
            newline.append("Review")
        IUV.append(newline)


with pd.ExcelWriter('Reports/10.IdentifiedUsers.xlsx') as writer:
    IUDF = pd.DataFrame(IU,columns=['Type','Name','UserName','Admin?','OSIAccess','server','domain','RACF','CIPAccess','ApprovedRole'])
    IUDF.to_excel(writer, sheet_name='IdentifiedUsers',index=False)
    IUDF = pd.DataFrame(IUV,columns=['Type','Name','UserName','Admin?','OSIAccess','server','domain','RACF','CIPAccess','VC1','ApprovedRole',"VC2","Recommended_Action"])
    IUDF.to_excel(writer, sheet_name='Validation',index=False)
    IUNCDF = pd.DataFrame(IUNC,columns=['Type','Name','UserName','Admin?','OSIAccess','server','domain','RACF'])
    IUNCDF.to_excel(writer, sheet_name='NoCIP',index=False)
    NFEDDF = pd.DataFrame(NFED,columns=['Type','Name','UserName','Admin?','OSIAccess','server','domain','RACF'])
    NFEDDF.to_excel(writer, sheet_name='NotFoundInEmpDirectory',index=False)
    RoleMapping = pd.DataFrame(list(siemenssubs.items()),columns=['OSI_Role','Siemens_Role'])
    RoleMapping.to_excel(writer, sheet_name='RoleMapping', index=False)
    
print("Permissions Loaded to File")

Permissions Loaded to File


In [63]:
OSI_C = []
for file in glob.glob('uds/all_UDS/*'):
    df, server = GetUsers(file)
    df['Server'] = server
    df['Domain'] = server_dict.get(server)
    OSI = df.values.tolist()
    OSI_Consoles = [line for line in OSI if line[0]=='Console']
    for line in OSI_Consoles:
        OSI_C.append(line)

In [64]:
file = 'Analysis\Consoles\TMS_Client_SCCM_List.xlsx'
df = pd.read_excel(file, sheet_name='Sheet1',na_values = '')
TMS_SCCM = df.values.tolist()
tms_consoles = [item[0] for item in TMS_SCCM]

for line in OSI_C:
    if line[2] in tms_consoles:
        line.append("Yes")
    else:
        line.append("No")
    
file = 'Analysis\Consoles\Corp_Client_SCCM_List.xlsx'
df = pd.read_excel(file, sheet_name='Successfully installed',na_values = '')
Corp_SCCM = df.values.tolist()
corp_consoles = [item[0] for item in Corp_SCCM]


for line in OSI_C:
    if line[2] in corp_consoles:
        line.append("Yes")
    else:
        line.append("No")
        
with pd.ExcelWriter('Reports/11.IdentifiedConsoles.xlsx') as writer:
    df = pd.DataFrame(OSI_C,columns=['Type', 'Assigned To', 'Name', 'Admin?', 'Groups', 'Server', 'Domain','TMS_Console','Corp_Console'])
    df.to_excel(writer, sheet_name='IdentifiedConsoles',index=False)
    
print("Module Complete")

Module Complete


In [70]:
from IPython.display import display

domain = 'TMS'
server = 'pdac2'

for file in glob.glob('uds/all_UDS/*'):
    fname = file.split("\\")[-1]
    if "uds.xml" in fname and server in fname:
        hrdf, domain = GetPermsMatrix(file)
roles = hrdf.columns.to_list()[3:]
file = 'Analysis\Approved_Access_03.2021\Approved_RoleDomainMatrix.xlsx'
df = pd.read_excel(file, sheet_name='RoleDomainMatrix',na_values = '')
approvedroles = Grouper(df,server,'Yes')['Group'].values.tolist()
removeroles = list(set([r for r in roles if r not in approvedroles]))
addroles = list(set([r for r in approvedroles if r not in roles]))

file = 'Analysis\Approved_Access_03.2021\Approved_Access_Consolidated_Feedback.xlsx'
AL_RbR = []
ALDF = pd.read_excel(file, sheet_name='Feedback',na_values = '')
AL = ALDF.values.tolist()
for line in AL:
    line.append(str(line[4]+line[5]))
IUV_Target = [line for line in IUV if line[5]==server]
for line in IUV_Target:
    line.append(str(line[7]+line[4]))
Targets = list(set([line[-1] for line in IUV_Target]))
addusers = [line[0:6] for line in AL if line[-1] not in Targets]




with pd.ExcelWriter('Reports/12.SystemAccessCleanup.xlsx') as writer:
    ARDF = pd.DataFrame(addroles,columns=['Roles'])
    ARDF.to_excel(writer, sheet_name='Roles To Add',index=False)
    RRDF = pd.DataFrame(removeroles,columns=['Roles'])
    RRDF.to_excel(writer, sheet_name='Roles To Remove',index=False)
    ADDF = pd.DataFrame(addusers,columns=['Type','Name','UserName','Admin?','RACF','OSI Access'])
    ADDF['server'] = server
    ADDF['domain'] = domain
    ADDF.to_excel(writer, sheet_name='User Roles To Add',index=False)
    REDF = Grouper(Grouper(IUDF,'server',server),'Recommended_Action','Review')
    REDF.to_excel(writer, sheet_name='User Roles To Remove',index=False)
    IUNCDF = pd.DataFrame(IUNC,columns=['Type','Name','UserName','Admin?','OSIAccess','server','domain','RACF'])
    try:
        Target = Grouper(IUNCDF,'server',server)
        Target.to_excel(writer, sheet_name='Users To Remove',index=False)
    except:
        pass
    PAMTU = Grouper(Grouper(IUDF,'server',server),'Recommended_Action','True-up PAM')
    PAMTU.to_excel(writer, sheet_name='True-up PAM',index=False)

print("Process Complete")

Process Complete


In [71]:
print("Process Complete")

Process Complete
